In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *

In [2]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/cellphoneaccessory"

## Dataset

In [3]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
type(df_train)

pandas.core.frame.DataFrame

In [5]:
df_train.head()

,text,label,dstype,words,tokens
110215,"Good quality, after 3 charges doubles duration...",0,test,"[ \n , xxbos, xxfld, 1, xxmaj, good, quality, ...","[18, 19, 20, 17, 2, 58, 106, 6, 125, 140, 283,..."
16078,THIS THING IS NOT WORTH THE WAIT AS WELL AS AN...,1,test,"[ \n , xxbos, xxfld, 1, xxup, this, xxup, thin...","[18, 19, 20, 17, 13, 12, 13, 162, 13, 11, 13, ..."
45386,This case looks great but i dont like the fact...,1,train,"[ \n , xxbos, xxfld, 1, xxmaj, this, case, loo...","[18, 19, 20, 17, 2, 12, 27, 148, 42, 29, 5, 49..."
63069,I've seen some reviews that complain about the...,0,train,"[ \n , xxbos, xxfld, 1, i, 've, seen, some, re...","[18, 19, 20, 17, 5, 115, 659, 107, 365, 21, 11..."
96949,These are incredible and install without fuss ...,0,test,"[ \n , xxbos, xxfld, 1, xxmaj, these, are, inc...","[18, 19, 20, 17, 2, 88, 40, 1299, 8, 445, 147,..."


In [6]:
df=pd.concat([df_train,df_valid],axis=0)

In [7]:
df['dstype'].value_counts()

train    86500
test     86500
Name: dstype, dtype: int64

In [8]:
df_train=df.loc[df['dstype']!='test']
df_test=df.loc[df['dstype']=='test']

In [9]:
df_train.shape, df_test.shape

((86500, 5), (86500, 5))

In [10]:
df_train.head()

,text,label,dstype,words,tokens
45386,This case looks great but i dont like the fact...,1,train,"[ \n , xxbos, xxfld, 1, xxmaj, this, case, loo...","[18, 19, 20, 17, 2, 12, 27, 148, 42, 29, 5, 49..."
63069,I've seen some reviews that complain about the...,0,train,"[ \n , xxbos, xxfld, 1, i, 've, seen, some, re...","[18, 19, 20, 17, 5, 115, 659, 107, 365, 21, 11..."
3441,The charger came with all of the correct Motor...,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, the, charger, c...","[18, 19, 20, 17, 2, 4, 78, 200, 25, 55, 14, 4,..."
122285,MediaDevil is my *go-to* screen protector. You...,0,train,"[ \n , xxbos, xxfld, 1, mediadevil, is, my, *,...","[18, 19, 20, 17, 5672, 11, 16, 280, 160, 36, 1..."
22017,i purchased this and loved it.. its so pretty ...,0,train,"[ \n , xxbos, xxfld, 1, i, purchased, this, an...","[18, 19, 20, 17, 5, 248, 12, 8, 569, 7, 322, 1..."


In [11]:
type(df_train['tokens'])

pandas.core.series.Series

In [12]:
bs=52 # 52 - Jeremey, 20 - default

In [13]:
bptt= 70 #70 - Jeremey, 35 - default

In [14]:
df_train.head()

,text,label,dstype,words,tokens
45386,This case looks great but i dont like the fact...,1,train,"[ \n , xxbos, xxfld, 1, xxmaj, this, case, loo...","[18, 19, 20, 17, 2, 12, 27, 148, 42, 29, 5, 49..."
63069,I've seen some reviews that complain about the...,0,train,"[ \n , xxbos, xxfld, 1, i, 've, seen, some, re...","[18, 19, 20, 17, 5, 115, 659, 107, 365, 21, 11..."
3441,The charger came with all of the correct Motor...,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, the, charger, c...","[18, 19, 20, 17, 2, 4, 78, 200, 25, 55, 14, 4,..."
122285,MediaDevil is my *go-to* screen protector. You...,0,train,"[ \n , xxbos, xxfld, 1, mediadevil, is, my, *,...","[18, 19, 20, 17, 5672, 11, 16, 280, 160, 36, 1..."
22017,i purchased this and loved it.. its so pretty ...,0,train,"[ \n , xxbos, xxfld, 1, i, purchased, this, an...","[18, 19, 20, 17, 5, 248, 12, 8, 569, 7, 322, 1..."


In [15]:
df_train['label'].value_counts()

0    74329
1    12171
Name: label, dtype: int64

In [16]:
df_valid['label'].value_counts()

0    14873
1     2427
Name: label, dtype: int64

In [17]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', '.', 'the']

In [18]:
padlen=1400
padding_idx=1

In [19]:
df_train['n_tok']=df_train['tokens'].apply(len)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_train['n_tok'].describe()

count    86500.000000
mean       116.635734
std        171.919658
min          5.000000
25%         38.000000
50%         62.000000
75%        125.000000
max       5867.000000
Name: n_tok, dtype: float64

In [21]:
df_train.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [23]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
1
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
1
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400
[18. 19. 20. 17. ...  1.  1.  1.  1.]
0
1400


In [25]:
df_train.shape

(86500, 6)

In [26]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(0    74329
 1    12171
 Name: label, dtype: int64, 0    14873
 1     2427
 Name: label, dtype: int64, 0    74328
 1    12172
 Name: label, dtype: int64)

In [27]:
df_train.shape, df_valid.shape

((86500, 6), (17300, 6))

In [28]:
padlen

1400

In [29]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [30]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [31]:
padlen

1400

In [32]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [33]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [34]:
for xb,yb,xlen in dltrain:
    break

In [35]:
xb, yb, xlen

(tensor([[18, 19, 20,  ...,  1,  1,  1],
         [18, 19, 20,  ...,  1,  1,  1],
         [18, 19, 20,  ...,  1,  1,  1],
         ...,
         [18, 19, 20,  ...,  1,  1,  1],
         [18, 19, 20,  ...,  1,  1,  1],
         [18, 19, 20,  ...,  1,  1,  1]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 tensor([ 77,  98, 353,  61,  92,  32,  34, 206, 183,  64, 183, 174,  33,  38,
         158, 351,  74,  70, 452,  27,  48,  69, 142,  66,  73,  68, 396,  45,
          31, 100, 143,  28,  52, 214,  67, 221,  32,  24,  83,  44, 255,  37,
          51,  83,  65,  85,  36,  58,  83,  58,  54, 176]))

## Model Architecture

In [36]:
from torch import nn
import torch
from torch.autograd import Variable

In [37]:
bs,bptt

(52, 70)

In [38]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.2 # 0.5 - changing to 0.4, 0.3 or any dropout value did not make much difference
dropout_o=0.5 #0.5
n_out=1

In [39]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [40]:
df_train['label'].value_counts()

0    74329
1    12171
Name: label, dtype: int64

In [41]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [42]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', '.', 'the']

In [43]:
import torch

In [44]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [45]:
m(input).size()

torch.Size([1, 64, 5])

In [46]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters+self.n_hidden*4,self.n_out)
        

        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs)) 
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)
        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        
        ## Concatenate
        big_out=torch.cat([cat_cnn,hidden,max_pool],dim=1)
        preds=self.fc(big_out)

        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))

        
        return preds.view(-1),loss
        
    

In [47]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [48]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [49]:
device="cpu"

## Load pretrained model weights

In [52]:
pretrained_lm_weights=pickle.load(open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights_unfreeze','rb'))

### Check if model forward works

In [53]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(30238, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [54]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (30238, 400)


In [55]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 18,990,201 trainable parameters


In [56]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(30238, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=1900, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [57]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.1038, -0.1617,  0.4763,  0.1031,  0.5394, -0.0359, -0.0891, -0.0819,
          0.0914,  0.2321, -0.1753,  0.1504,  0.1881,  0.0576,  0.1359,  0.3349,
         -0.0198,  0.0853,  0.0948,  0.0360,  0.2663, -0.1731,  0.0191,  0.0171,
         -0.1178,  0.2249,  0.2731,  0.0598,  0.1792,  0.1288, -0.0159, -0.0210,
          0.1913, -0.2608, -0.0144, -0.3651,  0.0881, -0.0331,  0.0551, -0.0612,
         -0.0864,  0.2334,  0.1120, -0.0632,  0.0744, -0.1690,  0.1217,  0.3580,
          0.1299,  0.0166,  0.0316,  0.0595], grad_fn=<ViewBackward>),
 tensor(0.7294, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [58]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [59]:
preds.view(-1)

tensor([ 0.0186, -0.0136, -0.0394,  0.3853,  0.3903,  0.0111,  0.1027, -0.4606,
         0.3577,  0.1939,  0.4739,  0.0445,  0.0780, -0.0223, -0.0019,  0.0445,
         0.1595,  0.2407,  0.2614,  0.2101,  0.0490, -0.1715, -0.1226,  0.1104,
         0.1200, -0.2724, -0.3210,  0.1809,  0.1269,  0.1004,  0.1300,  0.0332,
         0.1341,  0.1872, -0.0517, -0.3960, -0.0151,  0.0647, -0.0291, -0.0498,
         0.0244,  0.1217,  0.0850,  0.1081, -0.1186,  0.4414, -0.0540, -0.2426,
        -0.0486, -0.1957,  0.0390,  0.2643], grad_fn=<ViewBackward>)

In [60]:
yb

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [61]:
preds.size()

torch.Size([52])

In [62]:
yb.size()

torch.Size([52])

In [63]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.3846)

In [64]:
from sklearn.metrics import roc_auc_score

In [65]:
roc_auc_score(yb,preds)

0.5782312925170068

In [66]:
pretrained_lm_weights

array([[-0.096664,  0.068908, -0.026539,  0.145172, ...,  0.017248,  0.058272, -0.04472 , -0.068174],
       [-0.073669, -0.021713,  0.121824, -0.061521, ..., -0.144361,  0.072465, -0.00783 , -0.0527  ],
       [ 0.016913, -0.092459, -0.205166,  0.193297, ...,  0.012541,  0.087248,  0.136005,  0.118961],
       [ 0.151758,  0.590315,  0.002815,  0.07636 , ...,  0.053235,  0.03477 , -0.463683,  0.170703],
       ...,
       [-0.076562, -0.043897,  0.025359, -0.080075, ..., -0.00668 ,  0.143226,  0.024018, -0.079905],
       [-0.057223,  0.01818 ,  0.105571, -0.002987, ..., -0.118653,  0.09541 ,  0.104183,  0.028903],
       [-0.083589, -0.046181,  0.077349, -0.037635, ..., -0.14745 ,  0.130461,  0.051213, -0.123047],
       [-0.072408,  0.012531,  0.070828, -0.003311, ..., -0.061846,  0.097159, -0.050633, -0.060115]], dtype=float32)

## Learner

In [67]:
# Weight Decay Schedule
tempstart=1e-7
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*6

0 1e-07
1 6e-07
2 3.6e-06
3 2.16e-05
4 0.0001296
5 0.0007776


In [68]:
tempstart=5e-3
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*(0.7**i)

0 0.005
1 0.005
2 0.0034999999999999996
3 0.0017149999999999995
4 0.0005882449999999997
5 0.0001412376244999999


In [69]:
torch.randint(1,5,(1,))

tensor([1])

In [70]:
lrs=[1e-2,5e-3,1e-4,5e-4]

In [71]:
torch.randint(0,4,(1,))

tensor([2])

In [72]:
lrs[torch.randint(0,4,(1,))]

0.0001

In [73]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None,\
                 cycle_mult=0,lr_decay=0.7,wd_mult=6):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
        self.cycle_mult,self.lr_decay=cycle_mult,lr_decay
        self.wd_mult=wd_mult
        for param_group in self.optimizer.param_groups:
            self.start_lr=param_group['lr']
            self.start_wd=param_group['weight_decay']
        self.wd=self.start_wd
        self.lr=self.start_lr
        self.n_epoch=0
        self.lrs=[1e-2,5e-3,1e-4,5e-4]
        self.preds,self.preds_valid,self.trainY,self.actual=[],[],[],[]
        
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            
            if mode_train:
                self.trainY.append(Yb.view(-1))
                self.preds.append(preds.data)
            else:
                self.actual.append(Yb.view(-1))
                self.preds_valid.append(preds.data)
            
            del preds
        
        if mode_train:
            if 1==0:
                lr =self.lrs[torch.randint(0,4,(1,))]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

     
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        
        if self.cycle_mult > 0:
            reset_cycle=self.cycle_mult
        
        for epoch in range(n_epochs):                

            
            loss,acc=self.run_epoch(dltrain,True)
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Learning rate {self.lr} Weight Decay {self.wd} Train Loss:{loss} Train Accuracy:{acc} Valid Loss:{lossv} Valid Accuracy:{accv}')
        
            if self.cycle_mult:
                if self.n_epoch==reset_cycle:
                    self.lr=self.start_lr
                    #self.wd=self.start_wd
                    reset_cycle=self.n_epoch+reset_cycle
                else:
                    self.lr*=(self.lr_decay**self.n_epoch)  
                    if self.n_epoch>1:
                        self.wd*=self.wd_mult
            self.n_epoch+=1
                
                
            for param_group in self.optimizer.param_groups:
                param_group['lr']=self.lr
                #param_group['weight_decay']=self.wd

In [74]:
device="cuda:0"

In [75]:
n_layers

2

In [76]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (30238, 400)


In [77]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
#optimizer=torch.optim.SGD(model_sentiment.parameters(),lr=1e-2,momentum=0.9, weight_decay=wd)
metric_fn=accuracy_binomial

In [78]:
len(dltrain),len(dlvalid)

(1664, 333)

In [79]:
df_valid['label'].value_counts()

0    14873
1     2427
Name: label, dtype: int64

In [80]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,100,0.25,cycle_mult=6)

In [81]:
model_sentiment.freeze_embedding()

In [82]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [83]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [84]:
learner.run_epochs(dltrain,dlvalid,5)

Batch:100 0.41720510229468344  0.8544231115281582
Batch:200 0.36757185608148574  0.8652884986251592
Batch:300 0.3414945187419653  0.873076960593462
Batch:400 0.32591443171724677  0.8773077299818397
Batch:500 0.31097005707025527  0.8823846524655818
Batch:600 0.3038843103622397  0.8850000370293856
Batch:700 0.2969167406431266  0.8867308059547628
Batch:800 0.2894515492860228  0.8891827291809022
Batch:900 0.2839808928055896  0.8915171311133437
Batch:1000 0.27920361544191835  0.8932692681998015
Batch:1100 0.274783763885498  0.8949650723690337
Batch:1200 0.27080037764584025  0.8965224734321237
Batch:1300 0.2685747260714953  0.8972041794772332
Batch:1400 0.266121272378202  0.8982417956739664
Batch:1500 0.26354200233519076  0.8991923451324304
Batch:1600 0.26170703914016485  0.899867826057598
Batch:100 0.15108717687427997  0.9359615755081176
Batch:200 0.16316380670294164  0.9322115749120712
Batch:300 0.17952120245744785  0.9271154214938482
Epoch:0 Learning rate 0.005 Weight Decay 1e-07 Train Lo

In [85]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_freeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_freeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_freeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_freeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_freeze')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [114]:
from itertools import chain

In [156]:
preds=list(chain.from_iterable(learner.preds))[-df_train.shape[0]:]
preds_valid=list(chain.from_iterable(learner.preds_valid))[-df_valid.shape[0]:]
trainY=list(chain.from_iterable(learner.trainY))[-df_train.shape[0]:]
actual=list(chain.from_iterable(learner.actual))[-df_valid.shape[0]:]
preds=[x.item() for x in preds]
preds_valid=[x.item() for x in preds_valid]
trainY=[x.item() for x in trainY]
actual=[x.item() for x in actual]

In [157]:
from sklearn.metrics import roc_auc_score

In [158]:
roc_auc_score(trainY,preds),roc_auc_score(actual,preds_valid)

(0.993239042545457, 0.9871992982419396)

In [159]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [160]:
from scipy.special import expit

In [161]:
f1_score(trainY,np.round(expit(preds))), f1_score(actual,np.round(expit(preds_valid)))

(0.9074205537228046, 0.8904652618401836)

In [162]:
precision_score(trainY,np.round(expit(preds))), precision_score(actual,np.round(expit(preds_valid)))

(0.9226392663043478, 0.9019442096365173)

In [163]:
recall_score(trainY,np.round(expit(preds))), recall_score(actual,np.round(expit(preds_valid)))

(0.8926957521978474, 0.8792748248866914)

In [164]:
model_sentiment.unfreeze_embedding()

In [99]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [100]:
learner.lr

0.0001412376244999999

In [101]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,100,0.25,cycle_mult=0)

In [102]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:100 0.14735399410128594  0.9463461875915528
Batch:200 0.15444163320586085  0.9433654198050498
Batch:300 0.1546646469210585  0.9433333683013916
Batch:400 0.15534858888015152  0.9430288815498352
Batch:500 0.15498888851329684  0.942653881907463
Batch:600 0.15552975428911547  0.9419231126705806
Batch:700 0.15585355712633048  0.9417582782677242
Batch:800 0.15671150278998539  0.9409375364333391
Batch:900 0.15715933291241527  0.94074790014161
Batch:1000 0.15680831052549182  0.9405961909890175
Batch:1100 0.15661401318047535  0.9406468900225379
Batch:1200 0.15589223742485048  0.9407211905221144
Batch:1300 0.1559347337885545  0.9410059537795874
Batch:1400 0.15584843903779982  0.9411264101096563
Batch:1500 0.1548526171023647  0.9413333697319031
Batch:1600 0.15571187870111317  0.9410817673057318
Batch:100 0.11168353583663702  0.95923080265522
Batch:200 0.12000477449968457  0.9554808023571968
Batch:300 0.12496210868780812  0.9524359305699667
Epoch:0 Learning rate 0.0001412376244999999 Weight 

In [103]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_nofreeze')

In [113]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:100 0.1481987733580172  0.9457692664861679
Batch:200 0.1433498420752585  0.9475961899757386
Batch:300 0.14029900843277573  0.9484615739186605
Batch:400 0.1420077897561714  0.9473077285289765
Batch:500 0.14212058156356216  0.9470769592523575
Batch:600 0.14230550966225564  0.9469231126705805
Batch:700 0.1427850437563445  0.9467308045285089
Batch:800 0.14206708617042751  0.9467308043688536
Batch:900 0.1420172874712282  0.9468590092658996
Batch:1000 0.14130316203460097  0.9471346504092216
Batch:1100 0.1422005083665929  0.9469405943697149
Batch:1200 0.14250342370942234  0.9468429838120938
Batch:1300 0.14240365266799926  0.9468491475857221
Batch:1400 0.14232219084032943  0.9467857496227536
Batch:1500 0.14237443130960067  0.9464359328349431
Batch:1600 0.14247469146503136  0.9463341701403261
Batch:100 0.10694715581834316  0.9626923412084579
Batch:200 0.11476943859830499  0.9578846490383148
Batch:300 0.11936589508627851  0.955384649236997
Epoch:0 Learning rate 0.0001412376244999999 Weight

In [133]:
 torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_nofreeze')

In [134]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:100 0.11893330838531256  0.950961577296257
Batch:200 0.12474063745699823  0.9487500375509262
Batch:300 0.1246312196056048  0.9505769594510396
Batch:400 0.12652639192994683  0.9499519599974156
Batch:500 0.1260271285660565  0.9505384975671768
Batch:600 0.12753591419197619  0.9501923436919848
Batch:700 0.12659697566447514  0.9507692666564669
Batch:800 0.12775272281607614  0.9509375358372927
Batch:900 0.12872788649673264  0.9505342239141464
Batch:1000 0.1279005470518023  0.9507884972095489
Batch:1100 0.12823334855620155  0.950681853890419
Batch:1200 0.1281672568929692  0.9507692664861679
Batch:1300 0.12848263101938825  0.9508580239002521
Batch:1400 0.128067599464474  0.951126409471035
Batch:1500 0.12750687605142594  0.9512436254819234
Batch:1600 0.12742124902899377  0.9514183047786355
Batch:100 0.10042033202946186  0.9669231098890304
Batch:200 0.10634249138645828  0.9630769544839859
Batch:300 0.1108099785571297  0.9601923390229543
Epoch:0 Learning rate 0.0001412376244999999 Weight De

In [143]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_nofreeze')

In [144]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:100 0.10535536853596568  0.9559615725278854
Batch:200 0.10294018262065947  0.9585577276349068
Batch:300 0.10768594457457463  0.958012856443723
Batch:400 0.1106389448652044  0.95677888199687
Batch:500 0.1098244603537023  0.9571923427581787
Batch:600 0.11049984733108431  0.957596187988917
Batch:700 0.11104100002907216  0.9574450893912997
Batch:800 0.10949484838638454  0.9581490724533797
Batch:900 0.11059404889018172  0.9577778116861979
Batch:1000 0.11026997454557567  0.9581538800001145
Batch:1100 0.10925600338303908  0.9584790546243841
Batch:1200 0.1093431691483905  0.9585737514992555
Batch:1300 0.11000675562625895  0.9581805068254471
Batch:1400 0.11010531447960863  0.9582280555793218
Batch:1500 0.10955910892039537  0.9583590079545975
Batch:1600 0.10944370634679217  0.9583533990010619
Batch:100 0.09314460944384337  0.9698077249526977
Batch:200 0.0992743373569101  0.9657692623138427
Batch:300 0.10297813124954701  0.9641025952498118
Epoch:0 Learning rate 0.0001412376244999999 Weight 

In [154]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_nofreeze')

In [155]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:100 0.09336921118199826  0.9650000292062759
Batch:200 0.08921785645652562  0.9661538755893707
Batch:300 0.08965064291531841  0.9653205432494482
Batch:400 0.08891484803054482  0.9659134915471077
Batch:500 0.08503647636435926  0.9677307990789413
Batch:600 0.0868860703210036  0.9671795176466306
Batch:700 0.0869460022090269  0.9673351955413818
Batch:800 0.08787463970831595  0.9670192617923021
Batch:900 0.0877404782641679  0.9672222530841827
Batch:1000 0.08740715342666953  0.9671538771390915
Batch:1100 0.08672357464124533  0.9673951357061212
Batch:1200 0.08651272422401235  0.9673397746185461
Batch:1300 0.08623991446569562  0.9674408593544593
Batch:1400 0.08659317132950362  0.9674176136084965
Batch:1500 0.08648046600818635  0.9673846465746562
Batch:1600 0.08641796901123598  0.967283685207367
Batch:100 0.08448839819990099  0.9700000303983688
Batch:200 0.08841913866344839  0.9679808020591736
Batch:300 0.09339533329941332  0.9675641345977783
Epoch:0 Learning rate 0.0001412376244999999 Wei

In [165]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_model_dict_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo_optimizer_dict_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo_model_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo_optimizer_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo_learner_nofreeze')

In [166]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:100 0.06193555163219571  0.9755769520998001
Batch:200 0.06371829822193831  0.9755769520998001
Batch:300 0.06301001061064501  0.9767949000994365
Batch:400 0.06324502192204819  0.9769231052696705
Batch:500 0.06228198135923594  0.97719233584404
Batch:600 0.060421001594125605  0.9780448992053667
Batch:700 0.05941204830438697  0.9779945330108915
Batch:800 0.06033933809405426  0.9777884894609451
Batch:900 0.06036546212853864  0.977906010945638
Batch:1000 0.061074516123975624  0.9776346434354782
Batch:1100 0.06074349173693918  0.9779021256078373
Batch:1200 0.061080170981682995  0.9777243870000044
Batch:1300 0.06102376465802081  0.977633164204084
Batch:1400 0.060779998736605716  0.9775961818865367
Batch:1500 0.06098924499851031  0.9774487457672755
Batch:1600 0.06114639166633424  0.9773798353597521
Batch:100 0.07893657001201064  0.9744231045246124
Batch:200 0.08190254780696705  0.9720192605257034
Batch:300 0.08765519957523793  0.9714102852344513
Epoch:0 Learning rate 0.0001412376244999999